In [1]:
import os
import findspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as pd    
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/"
os.environ["SPARK_HOME"] = "/workspace/tripx/MCS/big_data/spark-3.1.1-bin-hadoop3.2"
findspark.init()
import time 
import logging
import seaborn as sns
from pyspark.ml.linalg import Vectors

import matplotlib.pyplot as plt
import pyspark.sql.functions as f
from pyspark.sql.functions import lit
from pyspark.sql.functions import col, count, explode, sum as sum_


In [2]:
# Stop spark if it existed
SparkContext.setSystemProperty('spark.executor.memory', '200g')

try:
    sc.stop()
except:
    print('sc have not yet created!')
sc = SparkContext(master = "local[*]", appName = "Multimodal Single Cell")
# sc = SparkContext.getOrCreate()
# Init session
print("Init session")
my_spark = SparkSession.builder.getOrCreate()
print(my_spark.catalog.listTables())

23/07/27 00:44:48 WARN Utils: Your hostname, lambda-server resolves to a loopback address: 127.0.1.1; using 192.168.1.252 instead (on interface eno1)
23/07/27 00:44:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/07/27 00:44:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


sc have not yet created!
Init session
[]


In [ ]:
sc._conf.getAll()

In [4]:
# train_cite_inputs_path = "/workspace/tripx/MCS/big_data/data/sub_sample_train_cite_inputs.csv"
# train_cite_inputs_data = my_spark.read.csv(train_cite_inputs_path,  header=True, maxColumns=30000)
# # train_cite_inputs_data.printSchema()

In [27]:
train_cite_target_path = "/workspace/tripx/MCS/big_data/data/train_cite_targets.csv"
train_cite_target_data = my_spark.read.csv(train_cite_target_path,  header=True)
# train_cite_target_data.printSchema()

In [6]:
# test_cite_inputs_path = "/workspace/tripx/MCS/big_data/data/sub_sample_test_cite_inputs.csv"
# test_cite_inputs_data = my_spark.read.csv(test_cite_inputs_path,  header=True,  maxColumns=30000)
# # test_cite_inputs_data.printSchema()

In [28]:
train_cite_target_data.createOrReplaceTempView('train_cite_target_data')

In [33]:
train_output = train_cite_target_data

In [35]:
new_cols=(column.replace('.', '_') for column in train_output.columns)
train_output = train_output.toDF(*new_cols)

In [36]:
from tqdm import tqdm

In [37]:
for col_name in tqdm(train_output.columns[1:]):
    # print(col_name)
    train_output = train_output.withColumn(col_name,  col(col_name).cast('float'))

  9%|▊         | 12/140 [00:00<00:01, 118.44it/s]

100%|██████████| 140/140 [00:01<00:00, 72.80it/s]


## Load extraced features

In [38]:
def zscore(x):
    x_zscore = []
    for i in range(x.shape[0]):
        x_row = x[i]
        x_row = (x_row - np.mean(x_row)) / np.std(x_row)
        x_zscore.append(x_row)
    x_std = np.array(x_zscore)    
    return x_std

In [39]:
feature_path='/dataset/NeurIPS2022/2nd-solution/kaggle/src_top2/senkin13/features/'

In [40]:
train_df = pd.read_feather(feature_path+'train_cite_inputs_id.feather')
test_df = pd.read_feather(feature_path+'test_cite_inputs_id.feather')

In [41]:
import numpy as np
cite_inputs_svd_clr = np.load(feature_path+'cite_inputs_svd_clr_200.npy')
train_cite_svd_clr = cite_inputs_svd_clr[:len(train_df)]
test_cite_svd_clr = cite_inputs_svd_clr[len(train_df):]
train_cite_svd_clr = zscore(train_cite_svd_clr)
test_cite_svd_clr = zscore(test_cite_svd_clr)

In [42]:
train_cite_svd_clr.shape

(70988, 200)

In [43]:
test_cite_svd_clr.shape

(48203, 200)

## Train

In [54]:
train_feature_df = map(lambda x: (int(1), Vectors.dense(x)), train_cite_svd_clr)
train_feature_df = my_spark.createDataFrame(train_feature_df,schema=["label", "features"])

In [55]:
train_feature_df.printSchema()

root
 |-- label: long (nullable = true)
 |-- features: vector (nullable = true)



In [57]:
train_data = train_feature_df.join(train_output)

In [60]:
train_data = train_data.drop('label')

In [61]:
train_data

DataFrame[features: vector, cell_id: string, CD86: float, CD274: float, CD270: float, CD155: float, CD112: float, CD47: float, CD48: float, CD40: float, CD154: float, CD52: float, CD3: float, CD8: float, CD56: float, CD19: float, CD33: float, CD11c: float, HLA-A-B-C: float, CD45RA: float, CD123: float, CD7: float, CD105: float, CD49f: float, CD194: float, CD4: float, CD44: float, CD14: float, CD16: float, CD25: float, CD45RO: float, CD279: float, TIGIT: float, Mouse-IgG1: float, Mouse-IgG2a: float, Mouse-IgG2b: float, Rat-IgG2b: float, CD20: float, CD335: float, CD31: float, Podoplanin: float, CD146: float, IgM: float, CD5: float, CD195: float, CD32: float, CD196: float, CD185: float, CD103: float, CD69: float, CD62L: float, CD161: float, CD152: float, CD223: float, KLRG1: float, CD27: float, CD107a: float, CD95: float, CD134: float, HLA-DR: float, CD1c: float, CD11b: float, CD64: float, CD141: float, CD1d: float, CD314: float, CD35: float, CD57: float, CD272: float, CD278: float, CD58

In [62]:
output_columns = train_output.columns[1:]

In [65]:
def train(lr_model, train_data, column_name):
    data_vector = train_data.select("features", column_name)
    data_vector = data_vector.withColumnRenamed(column_name, "label")
    # data_vector.show(5)
    # input()
    lr_model = lr.fit(data_vector)
    print("Coefficients: " + str(lr_model.coefficients))
    print("Intercept: " + str(lr_model.intercept))
    trainingSummary = lr_model.summary
    print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
    print("r2: %f" % trainingSummary.r2)

In [67]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='label', maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [68]:
for column_name in output_columns:
    train(lr, train_data, column_name)

23/07/27 01:05:26 WARN TaskSetManager: Stage 16 contains a task of very large size (1314 KiB). The maximum recommended task size is 1000 KiB.
23/07/27 01:05:26 WARN TaskSetManager: Stage 18 contains a task of very large size (1314 KiB). The maximum recommended task size is 1000 KiB.
23/07/27 01:05:28 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/07/27 01:05:28 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


KeyboardInterrupt: 

## Test

In [48]:
test_feature_df = map(lambda x: (int(1), Vectors.dense(x)), test_cite_svd_clr)
test_feature_df = my_spark.createDataFrame(test_feature_df,schema=["label", "features"])

In [49]:
test_feature_df.printSchema()

root
 |-- label: long (nullable = true)
 |-- features: vector (nullable = true)

